In [4]:
from PIL import Image, ImageDraw, ImageFont
import requests
import panel as pn
import openai
import textwrap
from io import BytesIO
import ast 

pn.extension(sizing_mode="stretch_width")
openai.api_key = ''
text_input = pn.widgets.TextInput(placeholder='A story about (try to be as descriptive as possible such as by giving your character a name)')
image_widget = pn.pane.Image()

def generate_text(num_pages):
    response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": 'Write a childrens book with %s pages. Each page should have a maximum of 50 words. Output the story in Python following this exact format: { "Page 1": "text", "Page 2": "text", "Page 3": "text" }. Make sure to avoid using line breaks anywhere in the story. No additional information or chit chat.' % (num_pages)},
                {"role": "user", "content": f'a story about {text_input.value}'},
            ],
            max_tokens=200
        )
    text_str = response['choices'][0]['message']['content']
    text_dict = ast.literal_eval(text_str)
    return text_dict, text_str

def replace_name(story):
    response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": 'Given a story. Try to remove all names and replacing it with the original noun. Noun isnt the actual name. Avoid using he and she. Output the story in Python following this exact format: { "Page 1": "text", "Page 2": "text", "Page 3": "text" }. Make sure to avoid using line breaks anywhere in the story. No additional information or chit chat.'},
                {"role": "user", "content": story},
            ],
            max_tokens=200
        )
    text = response['choices'][0]['message']['content']
    story_dict = ast.literal_eval(text)
    return story_dict


def generate_img(page, text):
    image = openai.Image.create(
        prompt=f"draw {text} in only 50 strokes",
        n=1,
        size="512x512"
    )
    image_url = image['data'][0]['url']
    request_result = requests.get(image_url)
    image = Image.open(BytesIO(request_result.content))
    

    # Define the new canvas size
    additional_width = image.width * 2 // 3  # Additional width on the right side
    new_width = image.width + additional_width

    # Create a new blank image with the expanded canvas size
    new_canvas = Image.new('RGB', (new_width, image.height), (255, 255, 255))  # RGB color tuple for white background

    new_canvas.paste(image, (0, 0))   
    draw = ImageDraw.Draw(new_canvas)
    
    # Define the font properties
    font_path = './CENSCBK.TTF'  # Path to your font file
    font_size = image.height // 36
    font_color = (0, 0, 0)  # RGB color tuple
    font = ImageFont.truetype(font_path, font_size)

    # Wrap the text within a specified width
    wrap_width = (5/3) * additional_width // (font_size)    # Adjust the wrap width based on the font size
    wrapped_text = textwrap.wrap(page, width=wrap_width)

    # Get the size of the wrapped text
    text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)

    # Calculate the position to center the text
    x = image.width + (additional_width - text_width) // 2
    y = (new_canvas.height - font_size * len(wrapped_text)) // 4

    # Overlay the wrapped text on the image
    for line in wrapped_text:
        draw.text((x, y), line, font=font, fill=font_color)
        y += 1.5 * text_height / len(wrapped_text)
    #image_widget.object = new_canvas
    return new_canvas


def generate_tabs(event):
    tabs_list = []
    pages, string = generate_text(pages_input.value)
    texts = replace_name(string)
    print(pages)
    print(texts)
    page_num = 1
    for page in pages:
        image = generate_img(pages[page], texts[page])  # Call the generate_img function to get the image
        image_pane = pn.pane.PNG(image)  # Create a PNG pane to display the image
        tab = pn.Column(name=f'Page {page_num}')  # Add the image pane to the column
        tab = pn.Column(image_pane, name=f'Page {page_num}')  # Add the image pane to the column
        tabs_list.append(tab)
        page_num += 1
    tabs.objects = tabs_list


pages_input = pn.widgets.TextInput(placeholder='Page Input', value='')
button_widget = pn.widgets.Button(name='Click Me')
button_widget.on_click(generate_tabs)
tabs = pn.Tabs()

template = pn.template.FastGridTemplate(
    title="AI Children Stories",
)
template.main[0, :7] = text_input
template.main[0, 7] = pages_input
template.main[0, 8:] = button_widget
template.main[1:5, 1:11] = tabs

# if alert_bool == True:
#     alert = pn.pane.Alert('AN error occured while generating your story. Try changing your prompt slight and submit again')
#     template.main[1, 1:11] = alert_bool



dashboard = pn.Column (
    '## Dashboard',
    pn.Row(text_input, pages_input, button_widget),
    tabs    
)
template.servable().show()

Launching server at http://localhost:64408


{'Page 1': 'James the black dog loved treats. He wagged his tail whenever he saw one. He would do tricks to get them. Treats made him happy.', 'Page 2': 'One day, James found a bag of treats in the park. He was so excited! But they were not his treats. James had to find their owner.', 'Page 3': 'James followed the scent until he found a little girl crying. Her name was Emily. James gave her the treats. He made a new friend.'}
{'Page 1': "The black dog loved treats. The black dog wagged the black dog's tail whenever the black dog saw one. The black dog would do tricks to get them. Treats made the black dog happy.", 'Page 2': "One day, the black dog found a bag of treats in the park. The black dog was so excited! But they were not the black dog's treats. The black dog had to find their owner.", 'Page 3': "The black dog followed the scent until the black dog found a little girl crying. The little girl's name was Emily. The black dog gave her the treats. The black dog made a new friend."}


C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)
C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)
C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)


{'Page 1': 'Once upon a time, there was a boy named Bob.', 'Page 2': 'Bob loved airplanes, big and small.', 'Page 3': 'Today, Bob and his dad went to the toy store.', 'Page 4': 'They found an amazing airplane toy, just for Bob.', 'Page 5': 'Bob was the happiest boy in the world!'}
{'Page 1': 'Once upon a time, there was a boy named [noun].', 'Page 2': '[noun] loved airplanes, big and small.', 'Page 3': 'Today, [noun] and [noun] went to the toy store.', 'Page 4': 'They found an amazing airplane toy, just for [noun].', 'Page 5': '[noun] was the happiest [noun] in the world!'}


C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)
C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)
C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize('\n'.join(wrapped_text), font)
C:\Users\impor\AppData\Local\Temp\ipykernel_14732\3527701267.py:73: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex